In [ ]:
import xarray as xr

import findlay2025a as f25a
import wisc_ecephys_tools as wet

In [ ]:
s3 = wet.get_sglx_project("shared")
nbsh = wet.get_sglx_project("seahorse")
experiment = wet.rats.constants.SleepDeprivationExperiments.NOD
sortings = f25a.units.get_nod_sortings()

In [ ]:
for subject, probes in sortings:
    print(f"\nDoing {subject}...")

    file = nbsh.get_experiment_subject_file(experiment, subject, "acg_metrics.pqt")
    # if file.exists():
    #    print(f"Skipping {subject} because metrics already exist")
    #    continue

    narrow_acgs, wide_acgs = f25a.units.load_narrow_and_wide_acgs(subject)

    # Get ACG fit
    fit_results = f25a.units.acg.get_fits(narrow_acgs)

    # Get ISI distribution shoulder
    smth_acgs = (
        narrow_acgs.rolling(time=3, center=True)
        .mean()
        .ffill(dim="time")
        .bfill(dim="time")
    )
    smth_isis = smth_acgs.sel(time=slice(0, None))
    shoulders = f25a.units.acg.get_shoulders(smth_isis)

    # Get burst index and moment
    burst_index = f25a.units.acg.get_burst_index(narrow_acgs, wide_acgs)
    moment = narrow_acgs.sel(time=slice(0, None)).mean(dim="time").rename("moment")

    # Merge metrics
    metrics = xr.merge(
        [
            fit_results,
            shoulders,
            burst_index,
            moment,
        ]
    )
    metrics = metrics.to_dataframe().reset_index()

    # Save metrics
    metrics.to_parquet(file)


Doing CNPIX2-Segundo...

Doing CNPIX3-Valentino...

Doing CNPIX4-Doppio...

Doing CNPIX5-Alessandro...

Doing CNPIX6-Eugene...

Doing CNPIX8-Allan...

Doing CNPIX9-Luigi...

Doing CNPIX10-Charles...

Doing CNPIX11-Adrian...

Doing CNPIX12-Santiago...


/home/gfindlay/projects/ece/seahorse/src/seahorse/units/acg.py:177: RuntimeWarning: invalid value encountered in scalar divide
  rsq = 1 - (ss_res / ss_tot)



Doing CNPIX14-Francis...

Doing CNPIX15-Claude...

Doing CNPIX17-Hans...

Doing CNPIX18-Pier...

Doing CNPIX19-Otto...

Doing CNPIX20-Ernst...
